In [12]:
import pinecone
import random
import itertools

## Connect to Pinecone Index

In [13]:
def initialize_pinecone():
    """
    To use Pinecone, one must have an API key.
    By default, ENVIRONMENT is us-west1-gcp.
    :return:None
    """
    PINECONE_API_KEY = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
    pinecone.init(api_key=PINECONE_API_KEY, environment="us-west1-gcp")


def connect_pinecone_index(pinecone_index_name):
    """
    Connection to the pinecone index
    :param pinecone_index_name
    :return: pinecone index
    """
    initialize_pinecone()
    pinecone_index = pinecone.Index(index_name=pinecone_index_name)
    return pinecone_index

In [14]:
initialize_pinecone()
pinecone_index_name = "myfirstindex"
index = connect_pinecone_index(pinecone_index_name)

## Prepare Dummy Data

In [42]:
def chunks(iterable, batch_size=100):
    """A helper function to break an iterable into chunks of size batch_size."""
    it = iter(iterable)
    chunk = tuple(itertools.islice(it, batch_size))
    while chunk:
        yield chunk
        chunk = tuple(itertools.islice(it, batch_size))

vector_dim = 512
vector_count = 100

# Example generator that generates many (id, vector) pairs
example_data_generator = map(lambda i: (f'id-{i}', [random.random() for _ in range(vector_dim)]), range(vector_count))

## Upsert Data

In [43]:
for ids_vectors_chunk in chunks(example_data_generator, batch_size=10):
    index.upsert(vectors=ids_vectors_chunk,namespace="my-first-namespace")

### 1. Check Records on Index

In [44]:
index.describe_index_stats()

{'dimension': 512,
 'index_fullness': 0.0,
 'namespaces': {'my-first-namespace': {'vector_count': 100}},
 'total_vector_count': 100}

### 2. Fetch Data by ID

In [49]:
fetch_response = index.fetch(ids=['id-0','id-9'],namespace="my-first-namespace")

### 3. Delete by ID

In [38]:
index.delete(ids=["id-1", "id-2"], namespace='my-first-namespace')

{}

### 4. Delete All Records

In [ ]:
index.delete(delete_all=True,namespace='my-first-namespace')

### 5. Delete by filter

In [56]:
index.delete(filter={"id": {"$eq": 'id-1'}},namespace="my-first-namespace")